<a href="https://colab.research.google.com/github/ykitaguchi77/FundusPhoto/blob/main/Metabo2024_tabular_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
print(f"CUDA is available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"Number of CUDA devices: {torch.cuda.device_count()}")

CUDA is available: True
CUDA version: 12.4
Number of CUDA devices: 1


In [4]:
import random
import timm
import copy
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from timm.scheduler import CosineLRScheduler
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import os
import glob
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_squared_error, r2_score
import time
import matplotlib.pyplot as plt
from torch.cuda.amp import GradScaler

%matplotlib inline

%cd "C:\Users\ykita\Metabo2024"

C:\Users\ykita\Metabo2024


In [5]:
# モデル枠組み読み込み
base_model = timm.create_model(model_name='swin_base_patch4_window12_384', num_classes=1, pretrained=False)

# GPU使用する場合
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
base_model = base_model.to(device)

# 学習済みモデル読み込み
model_path = 'model_20220903.pth'
#model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
base_model.load_state_dict(torch.load(model_path))



C:\Users\ykita\AppData\Local\Temp\ipykernel_21812\2652723574.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [ ]:
import os
import random
import copy
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import timm
from timm.scheduler.cosine_lr import CosineLRScheduler

# CUDA設定
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

class FundusDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None, use_tabular=True, tabular_features=None):
        self.data = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.use_tabular = use_tabular
        self.tabular_features = tabular_features

        if self.use_tabular and self.tabular_features is not None:
            self.tabular_data = self.data[self.tabular_features].values.astype(np.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = self.data.iloc[idx, 8]  # METSカラム

        if self.transform:
            image = self.transform(image)

        if self.use_tabular and self.tabular_features is not None:
            tabular = torch.tensor(self.tabular_data[idx])
        else:
            tabular = None

        return image, label, tabular


class FundusModel(nn.Module):
    def __init__(self, image_model, tabular_dim=7, num_classes=1, tabular_dropout_rate=0.3, final_dropout_rate=0.3):
        super(FundusModel, self).__init__()
        self.image_model = image_model
        in_features = self.image_model.head.in_features

        # タブラーブランチ（段階的にドロップアウト率を変更）
        self.tabular_fc = nn.Sequential(
            nn.Linear(tabular_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(tabular_dropout_rate)  # この値を段階的に上げていく
        )

        # 最終結合層（固定のドロップアウト率）
        self.final_fc = nn.Sequential(
            nn.Linear(in_features + 128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Dropout(final_dropout_rate),  # この値は0.3で固定
            nn.Linear(256, num_classes)
        )

    def forward(self, image, tabular=None):
        img_features = self.image_model.forward_features(image)
        img_features = self.image_model.global_pool(img_features)
        img_features = img_features.view(img_features.size(0), -1)

        if tabular is not None:
            tab_features = self.tabular_fc(tabular)
            combined = torch.cat((img_features, tab_features), dim=1)
        else:
            combined = torch.cat((img_features, torch.zeros(img_features.size(0), 128).to(image.device)), dim=1)

        out = self.final_fc(combined)
        return out

class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.best_model = None

    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            return True
        elif self.best_loss - val_loss > self.min_delta:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
            self.counter = 0
            return True
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
            return False

def train(model, train_loader, criterion, optimizer, device, scaler):
    model.train()
    running_loss = 0.0
    all_targets = []
    all_predictions = []

    for inputs, targets, tabular in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device).float().unsqueeze(1)
        if tabular is not None:
            tabular = tabular.to(device)

        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            outputs = model(inputs, tabular).squeeze(1)
            loss = criterion(outputs, targets.squeeze(1))

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()
        all_targets.extend(targets.cpu().numpy())
        all_predictions.extend(torch.sigmoid(outputs).detach().cpu().numpy())

    metrics = calculate_metrics(all_targets, all_predictions)
    metrics['loss'] = running_loss / len(train_loader)
    return metrics

def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_targets = []
    all_predictions = []

    with torch.no_grad():
        for inputs, targets, tabular in val_loader:
            inputs = inputs.to(device)
            targets = targets.to(device).float().unsqueeze(1)
            if tabular is not None:
                tabular = tabular.to(device)

            outputs = model(inputs, tabular).squeeze(1)
            loss = criterion(outputs, targets.squeeze(1))

            running_loss += loss.item()
            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(torch.sigmoid(outputs).cpu().numpy())

    metrics = calculate_metrics(all_targets, all_predictions)
    metrics['loss'] = running_loss / len(val_loader)
    return metrics

def calculate_metrics(all_targets, all_predictions):
    all_preds_binary = [1 if p >= 0.5 else 0 for p in all_predictions]
    return {
        'accuracy': accuracy_score(all_targets, all_preds_binary),
        'precision': precision_score(all_targets, all_preds_binary, zero_division=0),
        'recall': recall_score(all_targets, all_preds_binary, zero_division=0),
        'f1': f1_score(all_targets, all_preds_binary, zero_division=0),
        'auc': roc_auc_score(all_targets, all_predictions)
    }


def main():
    # シード固定
    seed_everything(42)

    # 保存先ディレクトリの設定と作成
    save_dir = r"C:\Users\ykita\Metabo2024\tabular_prediction"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
        print(f"Created directory: {save_dir}")

    # 元のデータの読み込み
    original_csv_path = "label_train.csv"
    original_df = pd.read_csv(original_csv_path)
    print(f"Original dataset size: {len(original_df)}")

    # 疾患データの読み込み
    disease_csv_path = 'metabo_disease.csv'
    disease_df = pd.read_csv(disease_csv_path)

    # 除外する疾患の定義
    exclude_conditions = [
        'AH', 'Blur', 'ERM', "Hemorrhage", "Coagulation",
        "VO", "Degeneration", "AMD", "CRA", "Drusen"
    ]

    # 除外するケースの抽出
    exclude_df = disease_df[disease_df['reason'].isin(exclude_conditions)]
    ah_blur_ids = exclude_df['id'].tolist()

    # クリーニング済みデータフレームの作成
    cleaned_df = original_df[~original_df['filename'].isin(ah_blur_ids)]
    print(f"Cleaned dataset size: {len(cleaned_df)}")

    # 表形式特徴量の定義
    tabular_features = ['age', 'AC', 'SBP', 'DBP', 'HDLC', 'TG', 'BS']

    # データの分割
    train_df, val_df = train_test_split(
        cleaned_df,
        test_size=0.2,
        random_state=42,
        stratify=cleaned_df['METS']
    )
    print(f"Training set size: {len(train_df)}")
    print(f"Validation set size: {len(val_df)}")

    # 表形式データの標準化
    scaler = StandardScaler()
    train_df[tabular_features] = scaler.fit_transform(train_df[tabular_features])
    val_df[tabular_features] = scaler.transform(val_df[tabular_features])

    # データ拡張の定義
    train_transform = transforms.Compose([
        transforms.Resize((384, 384)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_transform = transforms.Compose([
        transforms.Resize((384, 384)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # データセットの作成
    train_dataset = FundusDataset(
        dataframe=train_df,
        img_dir='cropped_images',
        transform=train_transform,
        tabular_features=tabular_features
    )

    val_dataset = FundusDataset(
        dataframe=val_df,
        img_dir='cropped_images',
        transform=val_transform,
        tabular_features=tabular_features
    )

    # データローダーの作成
    train_loader = DataLoader(
        train_dataset,
        batch_size=8,
        shuffle=True,
        num_workers=0,
        pin_memory=True
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        pin_memory=True
    )

    # 損失関数の定義
    criterion = nn.BCEWithLogitsLoss()

    # Gradient scaler for mixed precision training
    scaler = torch.cuda.amp.GradScaler()

    # ベースモデルの定義
    base_model = timm.create_model(
        model_name='swin_base_patch4_window12_384',
        num_classes=1,
        pretrained=False
    )

    # GPU使用する場合
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    base_model = base_model.to(device)

    # 学習済みモデル読み込み
    model_path = 'model_20220903.pth'
    base_model.load_state_dict(torch.load(model_path))

    # フェーズの設定
    phases = [
        {"name": "Phase1", "tabular_dropout": 0.3, "epochs": 200},
        {"name": "Phase2", "tabular_dropout": 0.5, "epochs": 200},
        {"name": "Phase3", "tabular_dropout": 0.7, "epochs": 200},
        {"name": "Phase4", "tabular_dropout": 0.8, "epochs": 200},
        {"name": "Phase5", "tabular_dropout": 0.9, "epochs": 200},
        {"name": "Phase6", "tabular_dropout": 0.95, "epochs": 200},
        {"name": "Phase7", "tabular_dropout": 1.0, "epochs": 200}
    ]

    FINAL_DROPOUT_RATE = 0.3
    base_metrics = None

    # メトリクスの記録用ファイル
    metrics_file = os.path.join(save_dir, "training_metrics.txt")
    with open(metrics_file, 'w') as f:
        f.write("Training started at: " + time.strftime("%Y-%m-%d %H:%M:%S") + "\n\n")
        f.write(f"Original dataset size: {len(original_df)}\n")
        f.write(f"Cleaned dataset size: {len(cleaned_df)}\n")
        f.write(f"Training set size: {len(train_df)}\n")
        f.write(f"Validation set size: {len(val_df)}\n")
        f.write(f"Excluded conditions: {', '.join(exclude_conditions)}\n\n")

    # フェーズごとのトレーニング
    for phase in phases:
        print(f"\nStarting {phase['name']} with tabular dropout rate {phase['tabular_dropout']}")
        print(f"Max epochs: {phase['epochs']}, Early stopping patience: 10")

        # モデルの初期化
        model = FundusModel(
            base_model,
            tabular_dim=len(tabular_features),
            tabular_dropout_rate=phase['tabular_dropout'],
            final_dropout_rate=FINAL_DROPOUT_RATE
        ).to(device)

        # 前のフェーズの重みがある場合はロード
        if phase != phases[0]:
            prev_model_path = os.path.join(save_dir, f'fundus_model_{phases[phases.index(phase)-1]["name"]}.pth')
            if os.path.exists(prev_model_path):
                model.load_state_dict(torch.load(prev_model_path))
                print(f"Loaded weights from: {prev_model_path}")

        # トレーニング設定
        optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
        scheduler = CosineLRScheduler(
            optimizer,
            t_initial=phase['epochs'],
            lr_min=1e-6,
            warmup_t=5,
            warmup_lr_init=1e-7,
            warmup_prefix=True
        )

        early_stopping = EarlyStopping(patience=10, min_delta=1e-4)

        # 現在のフェーズの保存パス
        save_path = os.path.join(save_dir, f'fundus_model_{phase["name"]}.pth')

        # フェーズごとのトレーニング実行
        metrics = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            scaler=scaler,
            early_stopping=early_stopping,
            num_epochs=phase['epochs'],
            device=device,
            save_path=save_path,
            phase_name=phase['name']
        )

        # メトリクスの記録
        if base_metrics is None:
            base_metrics = metrics

        # メトリクスをファイルに保存
        with open(metrics_file, 'a') as f:
            f.write(f"\n{phase['name']} Results:\n")
            f.write(f"Tabular Dropout Rate: {phase['tabular_dropout']}\n")
            f.write(f"Validation AUC: {metrics['auc']:.4f}\n")
            f.write(f"Relative to base phase: {(metrics['auc']/base_metrics['auc']-1)*100:.2f}% change\n")
            f.write("=" * 50 + "\n")

        print(f"\n{phase['name']} Final Results:")
        print(f"Validation AUC: {metrics['auc']:.4f}")
        print(f"Relative to base phase: {(metrics['auc']/base_metrics['auc']-1)*100:.2f}% change")
        print(f"Results saved to: {metrics_file}")
        print(f"Model saved to: {save_path}")

def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler,
                scaler, early_stopping, num_epochs, device, save_path, phase_name):
    best_epoch = 0
    history = {
        'train_loss': [], 'train_metrics': [],
        'val_loss': [], 'val_metrics': [],
        'lr': []
    }

    # トレーニング履歴の保存先
    history_file = os.path.join(os.path.dirname(save_path), f'history_{phase_name}.txt')

    print(f"\nStarting training for {phase_name}")
    print(f"Maximum epochs: {num_epochs}")
    print(f"Early stopping patience: {early_stopping.patience}")

    with open(history_file, 'w') as f:
        f.write(f"Training history for {phase_name}\n")
        f.write("Epoch,Train Loss,Train AUC,Val Loss,Val AUC,Learning Rate\n")

    for epoch in range(num_epochs):
        # トレーニング
        train_metrics = train(model, train_loader, criterion, optimizer, device, scaler)
        val_metrics = evaluate(model, val_loader, criterion, device)

        scheduler.step(epoch + 1)
        current_lr = optimizer.param_groups[0]['lr']

        # 結果の記録
        history['train_loss'].append(train_metrics['loss'])
        history['train_metrics'].append(train_metrics)
        history['val_loss'].append(val_metrics['loss'])
        history['val_metrics'].append(val_metrics)
        history['lr'].append(current_lr)

        # エポックごとの結果をファイルに保存
        with open(history_file, 'a') as f:
            f.write(f"{epoch+1},{train_metrics['loss']:.4f},{train_metrics['auc']:.4f},"
                   f"{val_metrics['loss']:.4f},{val_metrics['auc']:.4f},{current_lr:.2e}\n")

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train - Loss: {train_metrics['loss']:.4f}, AUC: {train_metrics['auc']:.4f}")
        print(f"Val - Loss: {val_metrics['loss']:.4f}, AUC: {val_metrics['auc']:.4f}")
        print(f"Learning Rate: {current_lr:.2e}")

        if early_stopping(val_metrics['loss'], model):
            best_epoch = epoch + 1
            if early_stopping.early_stop:
                print(f"\nEarly stopping triggered at epoch {epoch+1}")
                print(f"Best epoch was {best_epoch}")
                break

    # 最良モデルの保存
    model.load_state_dict(early_stopping.best_model)
    torch.save(model.state_dict(), save_path)

    print(f"\n{phase_name} Training Completed:")
    print(f"Best epoch: {best_epoch}/{epoch+1}")
    print(f"Training history saved to: {history_file}")
    print(f"Model saved to: {save_path}")

    return val_metrics

if __name__ == "__main__":
    main()

Original dataset size: 5000
Cleaned dataset size: 4618
Training set size: 3694
Validation set size: 924


C:\Users\ykita\AppData\Local\Temp\ipykernel_21812\3967310642.py:292: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
C:\Users\ykita\AppData\Local\Temp\ipykernel_21812\3967310642.py:307: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting 


Starting Phase1 with tabular dropout rate 0.3
Max epochs: 200, Early stopping patience: 10

Starting training for Phase1
Maximum epochs: 200
Early stopping patience: 10
